# 01 Spotify Ingestion, Cleaning, and SQLite Load

This notebook ingests exported Spotify Streaming History JSON files, normalizes
them into a canonical event-level schema, creates listening session identifiers,
and loads the results into a SQLite database table called `listening_events`.

Notes:
- This notebook handles Spotify data only. Apple Music ingestion will be added later.
- The objective is to create a clean, analysis-ready event table, not compute KPIs.

In [1]:
import pandas as pd
import sqlite3
import json
from pathlib import Path
from datetime import timedelta

## Step 1, Define paths and basic parameters

Expected Spotify export files:
- `data/raw/spotify/StreamingHistory*.json`

Output:
- SQLite database: `data/processed/MusicPlatformInsights.db`
- Table: `listening_events`

In [2]:
# Database path
DatabasePath = "../data/processed/MusicPlatformInsights.db"

# Raw Spotify data path
RawDataPath = Path("../data/raw/spotify")

# Spotify streaming history files
SpotifyFiles = list(RawDataPath.glob("StreamingHistory*.json"))

# Basic parameters
MinPlaySeconds = 30
SessionGapMinutes = 30

In [3]:
SpotifyFiles

[PosixPath('../data/raw/spotify/StreamingHistory_music_1.json'),
 PosixPath('../data/raw/spotify/StreamingHistory_music_0.json')]

## Step 2 — Validate files and load raw Spotify JSON

In this step, we verify that the expected Spotify export files exist and load
their contents into memory as raw listening event records.

In [4]:
# Find all Spotify streaming history JSON files in the raw data folder.
# This should pick up files like StreamingHistory_music_0.json, StreamingHistory_music_1.json, etc.
files = list(RawDataPath.glob("StreamingHistory*.json"))

# Quick sanity check to make sure the files were actually found
print("Files found:", len(files))

# Print a few file paths so it's obvious what is being loaded
for file in files[:5]:
    print(file)

# If no files are found, stop here.
# This usually means the path is wrong or the files weren't placed correctly.
if len(files) == 0:
    raise FileNotFoundError(
        "No Spotify StreamingHistory JSON files found in data/raw/spotify/")

Files found: 2
../data/raw/spotify/StreamingHistory_music_1.json
../data/raw/spotify/StreamingHistory_music_0.json


## Step 3, Load all JSON files into a single DataFrame

Spotify files are lists of listening events. We will stack them together into one dataset.

In [5]:
# List to hold a DataFrame for each Spotify JSON file
SpotifyFrames = []

# Loop through each Spotify streaming history file
for file in files:
    print(f"Loading {file}...")

    # Open the JSON file
    with open(file, "r", encoding="utf-8") as f:
        # json.load(f) returns a list of listening event dictionaries
        # pd.DataFrame(...) converts that list into a DataFrame
        SpotifyFrames.append(pd.DataFrame(json.load(f)))

# Concatenate all individual DataFrames into a single DataFrame
# ignore_index=True resets the row index so it is continuous
spotify = pd.concat(SpotifyFrames, ignore_index=True)

# Quick sanity check
print("Rows:", len(spotify))
spotify.head()

Loading ../data/raw/spotify/StreamingHistory_music_1.json...
Loading ../data/raw/spotify/StreamingHistory_music_0.json...
Rows: 15549


,endTime,artistName,trackName,msPlayed
0,2025-07-02 14:01,A$AP Rocky,Sundress,720
1,2025-07-02 14:01,Edith Whiskers,Home,650
2,2025-07-02 14:01,jagger finn,Vas,970
3,2025-07-02 14:01,JENNIE,Love Hangover (feat. Dominic Fike),78990
4,2025-07-02 14:01,Laufey,James,780


## Step 4 — Rename columns to canonical schema and clean types

Spotify exports use platform-specific field names. In this step, we rename those
columns into a canonical event schema and clean up data types so the data is
consistent and analysis-ready.

Spotify fields we expect:
- endTime
- artistName
- trackName
- msPlayed

Canonical fields:
- event_time
- platform
- artist
- track
- duration_minutes

In [6]:
# Rename Spotify-specific column names to match the canonical event schema
spotify = spotify.rename(columns={
    "endTime": "event_time",
    "artistName": "artist",
    "trackName": "track",
    "msPlayed": "ms_played"
})

# Parse the event timestamp into a datetime
# Use errors="coerce" to safely handle any malformed timestamps
spotify["event_time"] = pd.to_datetime(spotify["event_time"], errors="coerce")

# Spotify timestamps are naive — assume they are local time
# Convert to UTC for consistency with Apple Music
spotify["event_time"] = (
    spotify["event_time"]
    .dt.tz_localize("America/Los_Angeles", ambiguous="NaT", nonexistent="NaT")
    .dt.tz_convert("UTC"))

# Explicitly tag the platform so this can be combined with Apple Music later
spotify["platform"] = "spotify"

# Convert milliseconds played into minutes for easier interpretation
spotify["duration_minutes"] = spotify["ms_played"] / 1000 / 60

# Filter out very short plays (likely skips)
# MinPlaySeconds is defined earlier, so convert it to milliseconds here
spotify = spotify[spotify["ms_played"] >= (MinPlaySeconds * 1000)]

# Keep only the canonical fields for now
# Additional columns can be added later if needed
spotify = spotify[[
    "event_time",
    "platform",
    "artist",
    "track",
    "duration_minutes"
]]

# Sort events chronologically to prepare for session logic later
spotify = spotify.sort_values("event_time")

# Quick sanity check on the cleaned dataset
print("Clean rows:", len(spotify))
spotify.head()

Clean rows: 4089


,event_time,platform,artist,track,duration_minutes
5549,2024-08-10 02:45:00+00:00,spotify,Miguel,coffee,3.026717
5552,2025-01-08 11:54:00+00:00,spotify,Travis Scott,ASTROTHUNDER,2.382817
5578,2025-01-08 12:00:00+00:00,spotify,SZA,Another Life,2.880433
5593,2025-01-08 12:03:00+00:00,spotify,BigXthaPlug,Change Me,2.281100
5594,2025-01-08 12:07:00+00:00,spotify,The Marías,Heavy,4.220217


## Step 5 — Quick sanity checks

Before moving on, we run a few basic checks to make sure the cleaned data looks
reasonable and nothing obvious went wrong during ingestion.

We check:
- overall date range
- missing values in key fields
- basic listening duration statistics

In [7]:
# Check overall date range
print("Date range:")
print(spotify["event_time"].min(), "to", spotify["event_time"].max())

print("\nNull counts:")
print(spotify[["event_time", "artist", "track", "duration_minutes"]].isna().sum())

print("\nDuration (minutes) summary:")
spotify["duration_minutes"].describe()

Date range:
2024-08-10 02:45:00+00:00 to 2025-11-07 09:22:00+00:00

Null counts:
event_time          0
artist              0
track               0
duration_minutes    0
dtype: int64

Duration (minutes) summary:


count    4089.000000
mean        2.630969
std         1.209056
min         0.500033
25%         1.660317
50%         2.715033
75%         3.490667
max         8.495633
Name: duration_minutes, dtype: float64

## Step 6, Create session IDs

Definition:
- A new session starts when the gap between consecutive events is greater than `SessionGapMinutes`.

In [8]:
# Look at the timestamp of the previous listening event
# This makes it easier to compute gaps between plays
spotify["prev_time"] = spotify["event_time"].shift()

# Calculate the gap (in minutes) between the current event and the previous one
spotify["gap_minutes"] = (
    spotify["event_time"] - spotify["prev_time"]
).dt.total_seconds() / 60

# Start a new session if:
# - this is the first event (no previous time), or
# - the gap since the last event exceeds the session threshold
spotify["new_session"] = (
    spotify["gap_minutes"].isna() |
    (spotify["gap_minutes"] > SessionGapMinutes)
)

# Assign session IDs by cumulatively summing new session flags
spotify["session_id"] = spotify["new_session"].cumsum()

# Drop intermediate columns used only for session construction
spotify = spotify.drop(columns=["prev_time", "gap_minutes", "new_session"])

# Quick check
spotify.head()

,event_time,platform,artist,track,duration_minutes,session_id
5549,2024-08-10 02:45:00+00:00,spotify,Miguel,coffee,3.026717,1
5552,2025-01-08 11:54:00+00:00,spotify,Travis Scott,ASTROTHUNDER,2.382817,2
5578,2025-01-08 12:00:00+00:00,spotify,SZA,Another Life,2.880433,2
5593,2025-01-08 12:03:00+00:00,spotify,BigXthaPlug,Change Me,2.281100,2
5594,2025-01-08 12:07:00+00:00,spotify,The Marías,Heavy,4.220217,2


## Step 7 — Create SQLite database and table

All normalized listening events are stored in a single SQLite table:

- `listening_events`

Spotify data is loaded first and initializes the table.
Apple Music data will later append additional rows to the same table.

In [9]:
# Make sure the processed data directory exists before creating the database
# This avoids errors if the folder hasn't been created yet
Path("../data/processed").mkdir(parents=True, exist_ok=True)

# Open a connection to the SQLite database
# If the database file doesn't exist, SQLite will create it
connect = sqlite3.connect(DatabasePath)
cursor = connect.cursor()

# Explicitly create the listening_events table if it doesn't already exist
# This defines the schema we expect all platforms (Spotify, Apple Music) to follow
cursor.execute("""
CREATE TABLE IF NOT EXISTS listening_events (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_time TEXT,
    platform TEXT,
    artist TEXT,
    track TEXT,
    duration_minutes REAL,
    session_id INTEGER
);
""")

# Commit the table creation and close the connection
connect.commit()
print("Table ready: listening_events")

Table ready: listening_events


## Step 8 — Load Spotify events into SQLite

For Spotify-first ingestion, we replace any existing contents of the
`listening_events` table. This provides a clean reset while iterating on
the Spotify pipeline.

Apple Music events will be appended later once the Spotify flow is finalized.

In [10]:
# Write the cleaned Spotify events to the SQLite database
# Using `replace` here resets the table while iterating on the Spotify pipeline
spotify.to_sql(
    "listening_events",
    connect,
    if_exists="replace",
    index=False
)

# Commit the transaction to make sure changes are saved to disk
connect.commit()

# Quick confirmation prints
print("Loaded rows:", len(spotify))
print("Database:", DatabasePath)

# Close the database connection
connect.close()

Loaded rows: 4089
Database: ../data/processed/MusicPlatformInsights.db


## Step 9 — Verify rows exist in SQLite

As a final check, we run a simple SQL query to confirm that the
`listening_events` table exists and is populated as expected.

In [11]:
# Reconnect to the SQLite database to verify the Spotify load
connect = sqlite3.connect(DatabasePath)

# Simple verification query:
# - confirm rows exist
# - check the overall date range for Spotify events
CheckQuery = """
SELECT 
    COUNT(*) AS RowCount,
    MIN(event_time) AS MinTime,
    MAX(event_time) AS MaxTime
FROM listening_events
WHERE platform = 'spotify';
"""

# Run the query and load the result into a DataFrame
verification = pd.read_sql_query(CheckQuery, connect)

# Close the connection once the check is complete
connect.close()

verification

,RowCount,MinTime,MaxTime
0,4089,2024-08-10 02:45:00+00:00,2025-11-07 09:22:00+00:00
